# 1. Preprocess-Stage1-Download-TrainData-Unzip

## Run name

In [1]:
import time
project_name = 'Google-LandMark-Rec2019'
step_name = '1-Preprocess-Stage1-Download-TrainData-Unzip'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google-LandMark-Rec2019_1-Preprocess-Stage1-Download-TrainData-Unzip_20190424-135041


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
train_tar_count = 500

print('train_tar_count:', train_tar_count)
print('cpu_amount: ', cpu_amount)

train_tar_count: 500
cpu_amount:  1


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os, sys, gc, math, shutil, zipfile, pickle, h5py, re, tarfile
import urllib
from tqdm import tqdm
import hashlib

## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

md5sum_folder = os.path.join(input_folder, 'md5sum')
if not os.path.exists(md5sum_folder):
    os.mkdir(md5sum_folder)
    print('create folder:', md5sum_folder)
    
train_tar_folder = os.path.join(input_folder, 'train_tar')
if not os.path.exists(train_tar_folder):
    os.mkdir(train_tar_folder)
    print('create folder:', train_tar_folder)
    
train_untar_folder = os.path.join(input_folder, 'train_untar')
if not os.path.exists(train_untar_folder):
    os.mkdir(train_untar_folder)
    print('create folder:', train_untar_folder)
    
all_image_folder = os.path.join(input_folder, 'all_image')
if not os.path.exists(all_image_folder):
    os.mkdir(all_image_folder)
    print('create folder:', all_image_folder)

create folder: /data/landmark-recognition-2019/input/all_image


## unzip tar and move files

In [5]:
%%time
for i in range(train_tar_count):
    train_tar_file = os.path.join(train_tar_folder, 'images_%03d.tar' % i)
    image_folder = os.path.join(train_untar_folder, 'images_%03d' % i)
    if os.path.exists(image_folder):
        print(train_tar_file, '-->', image_folder, 'skiped')
        continue
    print(train_tar_file, '-->', image_folder)
    if os.path.exists(train_tar_file):
        with tarfile.open(train_tar_file) as tar:
            tar.extractall(image_folder)
        os.remove(train_tar_file)

/data/landmark-recognition-2019/input/train_tar/images_000.tar --> /data/landmark-recognition-2019/input/train_untar/images_000 skiped
/data/landmark-recognition-2019/input/train_tar/images_001.tar --> /data/landmark-recognition-2019/input/train_untar/images_001 skiped
/data/landmark-recognition-2019/input/train_tar/images_002.tar --> /data/landmark-recognition-2019/input/train_untar/images_002 skiped
/data/landmark-recognition-2019/input/train_tar/images_003.tar --> /data/landmark-recognition-2019/input/train_untar/images_003 skiped
/data/landmark-recognition-2019/input/train_tar/images_004.tar --> /data/landmark-recognition-2019/input/train_untar/images_004 skiped
/data/landmark-recognition-2019/input/train_tar/images_005.tar --> /data/landmark-recognition-2019/input/train_untar/images_005 skiped
/data/landmark-recognition-2019/input/train_tar/images_006.tar --> /data/landmark-recognition-2019/input/train_untar/images_006 skiped
/data/landmark-recognition-2019/input/train_tar/images_

## create soft link

In [6]:
def get_all_images_from_folder(folder):
    result_arr = []
    sub_folder_name_arr = [os.path.join(folder, folder_name) for folder_name in os.listdir(folder)]
    stack = sub_folder_name_arr
    while(stack):
        grand_children = stack.pop(0)
        if grand_children.endswith('.jpg'):
            src_img = grand_children
            result_arr.append(src_img)
        if not os.path.isdir(grand_children):
            continue
        stack += [os.path.join(grand_children, folder_name) for folder_name in os.listdir(grand_children)]
    return result_arr
    
temp_image_arr = get_all_images_from_folder('/data/landmark-recognition-2019/input/train_untar/images_000')
print(len(temp_image_arr))
print(temp_image_arr[:3])

8266
['/data/landmark-recognition-2019/input/train_untar/images_000/0/0/0/000949bb3b6480cb.jpg', '/data/landmark-recognition-2019/input/train_untar/images_000/0/0/0/000de9ff6063d246.jpg', '/data/landmark-recognition-2019/input/train_untar/images_000/0/0/0/0003ff2731d20c2a.jpg']


In [7]:
%%time
image_count = 0
softlink_count = 0
for i in range(train_tar_count):
    image_folder = os.path.join(train_untar_folder, 'images_%03d' % i)
    if os.path.exists(image_folder):
        src_image_arr = get_all_images_from_folder(image_folder)
        for src_image_path in src_image_arr:
            image_name = src_image_path.split('/')[-1]
            target_image_path = os.path.join(all_image_folder, image_name)
#             print(src_image_path, target_image_path)
            image_count += 1
            if not os.path.exists(target_image_path):
#                 print(src_image_path, target_image_path)
                softlink_count += 1
                os.symlink(src_image_path, target_image_path)
                if softlink_count % 10000 == 0:
                    print(softlink_count, end='-\t')
            if image_count % 10000 == 0:
                print(image_count, end='+\t')


10000-	10000+	20000-	20000+	30000-	30000+	40000-	40000+	50000-	50000+	60000-	60000+	70000-	70000+	80000-	80000+	90000-	90000+	100000-	100000+	110000-	110000+	120000-	120000+	130000-	130000+	140000-	140000+	150000-	150000+	160000-	160000+	170000-	170000+	180000-	180000+	190000-	190000+	200000-	200000+	210000-	210000+	220000-	220000+	230000-	230000+	240000-	240000+	250000-	250000+	260000-	260000+	270000-	270000+	280000-	280000+	290000-	290000+	300000-	300000+	310000-	310000+	320000-	320000+	330000-	330000+	340000-	340000+	350000-	350000+	360000-	360000+	370000-	370000+	380000-	380000+	390000-	390000+	400000-	400000+	410000-	410000+	420000-	420000+	430000-	430000+	440000-	440000+	450000-	450000+	460000-	460000+	470000-	470000+	480000-	480000+	490000-	490000+	500000-	500000+	510000-	510000+	520000-	520000+	530000-	530000+	540000-	540000+	550000-	550000+	560000-	560000+	570000-	570000+	580000-	580000+	590000-	590000+	600000-	600000+	610000-	610000+	620000-	620000+	630000-	630000+	640000-	64

In [8]:
print('image_count:', image_count)
print('softlink_count:', softlink_count)
print('target_image_path:', target_image_path)

image_count: 4132914
softlink_count: 4132914
target_image_path: /data/landmark-recognition-2019/input/all_image/ffc66facf567766d.jpg


In [9]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 1946.5s
Google-LandMark-Rec2019_1-Preprocess-Stage1-Download-TrainData-Unzip_20190424-135041
